# Dance Gestures (Drafted)

Gestures in dance refer to movements made by the dancer to convey a particular meaning or feeling. They can be subtle or overt and may include movements of the hands, arms, head, and body. Gestures can be used to express emotions, tell stories, and enhance the overall expressive power of the dance.

In this notebook, we are examining [Emma Joubert's Skinny Love (Birdy cover) Contemporary Solo](https://www.youtube.com/watch?v=QOlSCBRmfWY) and using [Mediapipe](https://google.github.io/mediapipe/) to extract x,y,z landmarks using its Pose Estimation Model. Numerical approach will be used to detect classic dance movements. But for now, only basic poses will be used.

The kind of data we have from the data collection:
* **x-coordinate**: Refers to the horizontal axis in relation to the width of the video frame
* **y-coordinate**: Refers to the vertical axis in relation to the height of the video frame
* **z-coordinate**: Refers to the relative position of the subject from the starting point of the camera. A positive z-coordinate indicates that the subject appears closer or larger, while a negative z-coordinate indicates that the subject appears further or smaller in relation to the camera's starting position.
* **Video frame rate:** The video has a frame per second rate of 24.9, which we will assume as a constant frequency throughout the analysis.

## Import Module

In [4]:
import seaborn as sns
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import mpl_toolkits
from mpl_toolkits.mplot3d import Axes3D
import cv2

In [5]:
cap = cv2.VideoCapture('./data/interim/frames/contemporary_dance_solo_20230219225812_10_annotated.mp4')
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print(f"h: {h}, w: {w}")
cap.release()
df = pd.read_csv('./data/interim/landmarks/contemporary_dance_solo_20230219225812_10.csv')
df.set_index('frame', inplace=True)
df.describe()

h: 360, w: 640


nose_x       nose_y       nose_z  left_eye_inner_x  \
count  1990.000000  1990.000000  1990.000000       1990.000000   
mean      0.486674     0.508155    -0.162423          0.487884   
std       0.111592     0.212577     0.136738          0.111606   
min       0.192119     0.061740    -0.627850          0.195189   
25%       0.408852     0.288870    -0.233980          0.409464   
50%       0.504349     0.546586    -0.155921          0.506232   
75%       0.567602     0.703593    -0.092311          0.569830   
max       0.784867     0.930235     0.308887          0.781023   

       left_eye_inner_y  left_eye_inner_z   left_eye_x   left_eye_y  \
count       1990.000000       1990.000000  1990.000000  1990.000000   
mean           0.500838         -0.168122     0.488810     0.500389   
std            0.216101          0.136887     0.111415     0.216051   
min            0.070341         -0.627326     0.197119     0.071537   
25%            0.277931         -0.238312     0.409861     0.277303   
50%            0.540745         -0.161258     0.507247     0.541576   
75%            0.700248         -0.094924     0.570279     0.698725   
max            0.941686          0.319031     0.780852     0.942080   

        left_eye_z  left_eye_outer_x  ...  right_heel_x  right_heel_y  \
count  1990.000000       1990.000000  ...   1990.000000   1990.000000   
mean     -0.168128          0.489766  ...      0.478440      0.732438   
std       0.136886          0.111221  ...      0.139324      0.104578   
min      -0.627234          0.199223  ...      0.106904      0.293088   
25%      -0.238336          0.409687  ...      0.374376      0.683639   
50%      -0.161292          0.507716  ...      0.471068      0.737317   
75%      -0.094996          0.570869  ...      0.580412      0.783367   
max       0.319073          0.780556  ...      0.869751      0.974827   

       right_heel_z  left_foot_index_x  left_foot_index_y  left_foot_index_z  \
count   1990.000000        1990.000000        1990.000000        1990.000000   
mean       0.148964           0.513086           0.759403           0.056057   
std        0.157121           0.149041           0.106489           0.164449   
min       -0.420691           0.123174           0.182954          -0.510425   
25%        0.043600           0.377213           0.724959          -0.063745   
50%        0.132788           0.498428           0.778811           0.055392   
75%        0.263136           0.640521           0.804315           0.142438   
max        0.709645           0.873557           1.086050           0.612418   

       right_foot_index_x  right_foot_index_y  right_foot_index_z  landmarks  
count         1990.000000         1990.000000         1990.000000        0.0  
mean             0.472710            0.743136            0.117676        NaN  
std              0.149195            0.104481            0.174131        NaN  
min              0.076826            0.245948           -0.496661        NaN  
25%              0.355182            0.696789           -0.001820        NaN  
50%              0.461724            0.745325            0.104188        NaN  
75%              0.581855            0.797304            0.251002        NaN  
max              0.903751            0.974501            0.669081        NaN  

[8 rows x 100 columns]

In [6]:
df.loc[:, df.columns.str.endswith('_x')] *= w
df.loc[:, df.columns.str.endswith('_y')] *= h
df.loc[:, df.columns.str.endswith('_y')] = h - df.loc[:, df.columns.str.endswith('_y')]

# Understanding the Z-coordinate can help us determine the quality of the dance video
# If the mean average is close to 0, it may indicate the subject's position is consistent
z_cols = df.filter(regex='_z$')
z_min = round(z_cols.min().min(),2)
z_max = round(z_cols.max().max(),2)
z_mean = round(z_cols.mean().mean())
print(f"Smallest z-value: {z_min}, Largest z-value: {z_max}, Average: {z_mean}")

Smallest z-value: -0.66, Largest z-value: 0.71, Average: 0


### Mediapipe's Post Landmark
![Image](https://mediapipe.dev/images/mobile/pose_tracking_full_body_landmarks.png)

### Defining Basic Poses

![Image](https://i.ibb.co/zFDftG3/poses.png)

Let's first define the basic positions using using a numerical approach

1. `Standing`: If the difference between the height of the right and left sides is less than 0.08

2. `Sitting down`: If the difference between the y-axis 

3. `Bent Knee`: If the angle from hip to knee not within the range of 85-95

## Numerical Operations
Numerical operations required to get joint estimates, distances, and angles using geometry

In [7]:
# Estimated forehead position
df.loc[:, 'forehead_x'] = (df['left_eye_outer_x'] + df['right_eye_outer_x']) / 2
df.loc[:, 'forehead_y'] = (df['left_eye_outer_y'] + df['right_eye_outer_y']) / 2
df.loc[:, 'forehead_z'] = (df['left_eye_outer_z'] + df['right_eye_outer_z']) / 2

# Estimated torso position
df.loc[:, 'torso_x'] = (df['left_shoulder_x'] + df['right_shoulder_x'] + df['left_hip_x'] + df['right_hip_x']) / 4
df.loc[:, 'torso_y'] = (df['left_shoulder_y'] + df['right_shoulder_y'] + df['left_hip_y'] + df['right_hip_y']) / 4
df.loc[:, 'torso_z'] = (df['left_shoulder_z'] + df['right_shoulder_z'] + df['left_hip_z'] + df['right_hip_z']) / 4

# Eucleadian distance between right shoulder to hip
df.loc[:, 'dis_right_shoulder_to_hip'] = np.sqrt((df['right_hip_y'] - df['right_knee_y'])**2)
df.loc[:, 'dis_left_shoulder_to_hip'] = np.sqrt((df['left_hip_y'] - df['left_knee_y'])**2)

# Eucleadian distance between hip to knee
df.loc[:, 'dis_right_hip_to_knee'] = np.sqrt((df['right_hip_y'] - df['right_knee_y'])**2)
df.loc[:, 'dis_left_hip_to_knee'] = np.sqrt((df['left_hip_y'] - df['left_knee_y'])**2)

# Eucleadian distance between knee to ankle
df.loc[:, 'dis_right_knee_to_ankle'] = np.sqrt((df['right_knee_y'] - df['right_ankle_y'])**2)
df.loc[:, 'dis_left_knee_to_ankle'] = np.sqrt((df['left_knee_y'] - df['left_ankle_y'])**2)

# Eucleadian distance between forehead to right shoulder
df.loc[:, 'dis_forehead_to_right_shoulder'] = np.sqrt((df['forehead_y'] - df['right_shoulder_y'])**2)
df.loc[:, 'dis_forehead_to_left_shoulder'] = np.sqrt((df['forehead_y'] - df['left_shoulder_y'])**2)

# Difference between height to the left and to the right
df.loc[:, 'total_height_r'] = df['dis_forehead_to_right_shoulder'] + df['dis_right_shoulder_to_hip'] + df['dis_right_hip_to_knee'] + df['dis_right_knee_to_ankle']
df.loc[:, 'total_height_l'] = df['dis_forehead_to_left_shoulder'] + df['dis_left_shoulder_to_hip'] + df['dis_left_hip_to_knee'] + df['dis_left_knee_to_ankle']
df.loc[:,'diff_height_r_and_l'] = (df['total_height_r'] - df['total_height_l']).abs()

# Difference in x-axis from hip to ankle
df.loc[:, 'diff_hip_to_ankle_r'] = (df['right_hip_x']-df['right_ankle_x']).abs()
df.loc[:, 'diff_hip_to_ankle_l'] = (df['left_hip_x']-df['left_ankle_x']).abs()

# calculate the angle between the hip and knee joint for the right leg
df['angle_hip_knee_r'] = df.apply(lambda row: math.degrees(math.atan2(row['right_knee_y'] - row['right_hip_y'], row['right_knee_x'] - row['right_hip_x'])), axis=1)
df['angle_hip_knee_l'] = df.apply(lambda row: math.degrees(math.atan2(row['left_knee_y'] - row['left_hip_y'], row['left_knee_x'] - row['left_hip_x'])), axis=1)

## Basic Poses Algorithms
This code estimates positions of different body parts, computes Euclidean distances between them, calculates the difference between heights, difference in x-axis from hip to ankle, and the angle between the hip and knee joint for the left and right legs for each row in the dataframe.

In [8]:
def measure_pose_standing(df, threshold):
    dfs = df.copy()
    dfs['diff_rate_r'] = dfs['diff_height_r_and_l'] / dfs['total_height_r']
    dfs['diff_rate_l'] = dfs['diff_height_r_and_l'] / dfs['total_height_l']
    dfs['both_legs_straight'] = (dfs['diff_rate_r'] <= threshold) & (dfs['diff_rate_l'] <= threshold)
    total = dfs['both_legs_straight'].sum()
    return round(total/24.9,2)

print(f"Estimated standing position {measure_pose_standing(df,0.08)} s")

def measure_pose_sitting(df, threshold):
    dfs = df.copy()
    dfs['diff_rate_r'] = dfs['diff_hip_to_ankle_r']/((dfs['right_hip_x']+dfs['right_ankle_x'])/2)
    dfs['diff_rate_l'] = dfs['diff_hip_to_ankle_r']/((dfs['right_hip_x']+dfs['right_ankle_x'])/2)
    dfs['both_legs_sitting'] = (dfs['diff_rate_r'] <= threshold) & (dfs['diff_rate_l'] <= threshold)
    total = dfs['both_legs_sitting'].sum()
    return round(total/24.9,2)

print(f"Estimated sitting position {measure_pose_sitting(df,0.08)} s")


def measure_pose_bent_knees(df):
    dfs = df[['angle_hip_knee_r','angle_hip_knee_l']].abs()
    dfs['bent_knees'] = (dfs['angle_hip_knee_r'] < 80) | (dfs['angle_hip_knee_r'] > 100) & (dfs['angle_hip_knee_l'] < 80) | (dfs['angle_hip_knee_l'] > 100)
    total = dfs['bent_knees'].sum()
    return round(total/24.9,2)

print(f"Estimated bent knee position {measure_pose_bent_knees(df)} s")

Estimated standing position 23.53 s
Estimated sitting position 37.55 s
Estimated bent knee position 59.36 s


## To be continued